## Callysto Course Link Checker 

**Description**: This is a notebook that iterates through directory files and runs a validation check on the links contained in Jupyter Notebooks. This notebook was written for Callysto internal use.

**Usage**: Run this notebook in the parent directory containing the notebooks or in the directory itself.

**Notes**

Last Edited: June 16, 2020

Author: LNC

Contact: lisa.cao@cybera.ca

In [28]:
# install cell run only once if needed
# !pip3 install urllib3

In [31]:
import os
import json
import re
import urllib3

In [30]:
## function to parse urls (from geeksforgeeks)
def url_parse(string): 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)       
    return [x[0] for x in url] 

In [48]:
# function to check for active urls
def web_status(cell_urls): 
    http = urllib3.PoolManager()
    for url in cell_urls: 
        http.request('GET', urls[i], timeout = 10.0)
        if req.status == 200:
            pass
        else: 
            error = urllib3.error.URLError
            print(error.reason)
        
            

In [ ]:
## search through all directories and parse cells
for root, dirs, files in os.walk("."):
    for filename in files:
        if filename.endswith('.ipynb'):
            notebook_name = filename[:-6]
            file = os.path.join(root, filename)
            notebook = json.load(open(file))
            cell_number = 0
            for cell in notebook['cells']:
                cell_number += 1
                ####
                cell_contents = cell['source'][0]
                cell_urls = url_parse(cell_contents)
                
                #### 
                if link_valid == FALSE: 
                    print(+notebook_name+":Cell"+cell_number+"contains broken links") #add url link?
                    ###
                    if link_valid == TRUE: 
                        
                    
                
                